In [70]:
#import libraries
import pandas as pd
import os
import shutil

In [23]:
# Read datasets
classes = pd.read_csv('../data/class-descriptions-boxable.csv')
class_labels_dict = { row['label']:row['name'].replace(' ','_') for _, row in classes.iterrows() }
class_labels_dict_index = { row['name'].replace(' ','_'):index for index, row in classes.iterrows() }
class_labels_dict_index

{'Human_body': 0,
 'Human_foot': 1,
 'Human_leg': 2,
 'Human_head': 3,
 'Human_arm': 4,
 'Human_hand': 5}

In [65]:
# Define function to reshape boxes to yolo format
def create_yolo_boxes(path, index, path_save = ''):
    for filename in os.listdir(path):
        if filename.endswith(".txt"):
            f = open(path + '/' + filename, "r")
            content = f.read()
            lines = content.split('\n')
            outputs = []
            for line in lines:
                if line == '':
                    continue
                elements = line.split(',')
                if len(elements) < 5:
                    continue
                    
                xmin,xmax,ymin,ymax = float(elements[1]),float(elements[2]),float(elements[3]),float(elements[4])
                cx = (xmin + xmax) / 2
                cy = (ymin + ymax) / 2
                w  = xmax-xmin
                h  = ymax-ymin
                outputs.append(str(index) + ' ' + str(cx) + ' ' + str(cy) + ' ' + str(w) + ' ' + str(h) + '\n')
            f.close()
            
            if not os.path.exists(path + '/' + path_save):
                os.mkdir(path + '/' + path_save)
            f = open(path + '/' + path_save + '/' + filename, "w")
            for o in outputs:
                f.write(o)
            f.close()
            

In [66]:
path = '../data/train/'
for key in class_labels_dict_index:
    create_yolo_boxes(path + key, class_labels_dict_index[key], 'labels/')

### Merge labels

In [75]:
set_labels = set()
for filename in os.listdir(path + 'labels/'):
    if filename.endswith(".txt"):
        set_labels.add(filename)

print(len(set_labels))
        
def append_labels(path, labels, destiny):
    for filename in os.listdir(path):
        if filename in labels:
            with open(destiny + '/' + filename, 'a') as outfile:
                with open(path + filename) as infile:
                    for line in infile:
                        outfile.write(line)
        else:
            shutil.copy(path + filename, destiny + filename)
            labels.add(filename)
    return labels

0


In [76]:
set_labels = append_labels(path + 'Human_arm/labels/', set_labels, path + 'labels/')
print(len(set_labels))

2073


In [77]:
set_labels = append_labels(path + 'Human_body/labels/', set_labels, path + 'labels/')
print(len(set_labels))

3994


In [78]:
set_labels = append_labels(path + 'Human_foot/labels/', set_labels, path + 'labels/')
print(len(set_labels))

4943


In [79]:
set_labels = append_labels(path + 'Human_hand/labels/', set_labels, path + 'labels/')
print(len(set_labels))

6714


In [80]:
set_labels = append_labels(path + 'Human_head/labels/', set_labels, path + 'labels/')
print(len(set_labels))

8413


In [81]:
set_labels = append_labels(path + 'Human_leg/labels/', set_labels, path + 'labels/')
print(len(set_labels))

10079


In [82]:
images = set()
def copy_images(origin, destiny, images):
    for filename in os.listdir(origin):
        if filename.endswith('.jpg'):
            if filename in images:
                continue
            else:
                shutil.copy(origin + filename, destiny + filename)
                images.add(filename)
            
    return images

In [83]:
for key in class_labels_dict_index:
    images = copy_images(path + key + '/', path + 'images/', images)
    print(len(images))

2042
2996
4845
6687
8439
10076


In [86]:
for label in set_labels:
    not_found = True
    for image in images:
        if label[:-3] == image[:-3]:
            not_found = False
            break
    if not_found:
        print(label)

49262b8f828e9d43.txt
b325b67fb67718de.txt
ba1ea6102b729c92.txt
